In [ ]:
%pip install google-cloud-translate==2.0.1

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Uninstalling protobuf-4.23.4:
      Successfully uninstalled protobuf-4.23.4
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.2
    Uninstalling cachetools-5.3.2:
      Successfully uninst

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth-oauthlib 1.2.0 requires google-auth>=2.15.0, but you have google-auth 1.35.0 which is incompatible.


In [2]:
import pandas as pd

df_test_efr = pd.read_json(f'EDiReF_test_data\MaSaC_test_efr.json')
df_test_erc = pd.read_json(f'EDiReF_test_data\MaSaC_test_erc.json')

df_train_efr = pd.read_json(f'EDiReF_train_data\MaSaC_train_efr.json')
df_train_erc = pd.read_json(f'EDiReF_train_data\MaSaC_train_erc.json')

df_val_efr = pd.read_json(f'EDiReF_val_data\MaSaC_val_efr.json')
df_val_erc = pd.read_json(f'EDiReF_val_data\MaSaC_val_erc.json')


In [8]:
df_test_efr_sent_count = len([sent for conv in list(df_test_efr['utterances']) for sent in conv])
df_test_erc_sent_count = len([sent for conv in list(df_test_erc['utterances']) for sent in conv])
df_train_efr_sent_count = len([sent for conv in list(df_train_efr['utterances']) for sent in conv])
df_train_erc_sent_count = len([sent for conv in list(df_train_erc['utterances']) for sent in conv])
df_val_efr_sent_count = len([sent for conv in list(df_val_efr['utterances']) for sent in conv])
df_val_erc_sent_count = len([sent for conv in list(df_val_erc['utterances']) for sent in conv])

print(df_test_efr_sent_count + df_test_erc_sent_count + df_train_efr_sent_count \
      + df_train_erc_sent_count + df_val_efr_sent_count + df_val_erc_sent_count)

125369


In [9]:
sentence = df_test_efr['utterances'][10][0]
print(sentence)

bina padhe tum wo paper raddi me kaise daal diye?


In [12]:
from google.cloud import translate_v2

translate_client = translate_v2.Client()
result = translate_client.translate(sentence, source_language="hi", target_language="en")
print(result)

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [ ]:
import pandas as pd
import yaml

dataset_types = ['test', 'train', 'val']
tasks = ['efr', 'erc']

for type in dataset_types:
    for task in tasks:
        df = pd.read_json(f'EDiReF_{type}_data\MaSaC_{type}_{task}.json')
        # yaml.safe_dump(df['utterances'].values.tolist(), open(f'masac_{type}_{task}.yml', 'w'))

In [ ]:
transaltion = yaml.safe_load(open('translated_output.yml'))
for i in transaltion['translations']:
    print(transaltion['translations'][i]['english'])

In [ ]:
sentence = df['utterances'][0][0]
print(sentence)

nehi nehi main rajsekhar inamdar


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
from pyhinavrophonetic import hinavro

sentence = "Aaj to bhot awful day tha!"

article = hinavro.parse(sentence)
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(**inputs, forced_bos_token_id = tokenizer.convert_tokens_to_ids("eng_Latn"), max_length = 30)

output = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(output)

Today was so full of joy!


In [ ]:
import ollama

response = ollama.chat(model='llama2', messages=[
  {
    'role': 'user',
    'content': f'Translate this sentence from Hindi to English: "{sentence}". Respond only with the translation itself, with no explanations, additional words, or punctuation outside of the translation.',
  },
])

print(response['message']['content'])


"Aaj to bhot awful day tha!" = "It's been a really bad day today!"
